In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [3]:
# train 데이터 가져오기
df_all = pd.DataFrame()
year = 2020
for k in range(2):
  forders = os.listdir('/content/drive/MyDrive/Meta Data/'+str(year)+'/')
  for i in range(0,len(forders)):
      if forders[i].split('.')[1] == 'csv':
          file = '/content/drive/MyDrive/Meta Data/'+str(year)+'/'+forders[i]
          # 검증 데이터셋인 202107은 제외시키자
          if forders[i]=='202107_uv.csv':
            continue
          df= pd.read_csv(file,encoding='utf-8')
          df_all = pd.concat([df_all, df],ignore_index=True)
  year+=1

# test 데이터 가져오기
df = pd.read_csv('/content/drive/MyDrive/Meta Data/2021/202107_uv.csv')

In [4]:
# band5,band8,band9,band10,band11,'band14','band15','band16',sateza 제외
scale_cols = ['band1', 'band2', 'band3','band4','band6','band7','band12','band13','solarza','esr'] # 0.3 이상

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from sklearn.metrics import confusion_matrix

# stn 값들
stn_index = [13,105,108,112,115,131,132]

# stn 별 예측한 값을 담을 배열
prediction = []
TP = []

# 오차범위
DIFF = 0.2

# stn 별 
for i in stn_index:
  # 학습 데이터 정규화
  df_scaled = scaler.fit_transform(df_all[df_all['stn']==i][scale_cols])
  df_scaled = pd.DataFrame(df_scaled)
  df_scaled.columns = scale_cols

  # 테스트 데이터 정규화
  df_scaled_test = scaler.fit_transform(df[df['stn']==i][scale_cols])
  df_scaled_test = pd.DataFrame(df_scaled_test)
  df_scaled_test.columns = scale_cols

  # 3차원으로 재구성하기
  X_train = df_scaled.values
  X_train_t = X_train.reshape(X_train.shape[0], X_train.shape[1],1) # 데이터셋 수, 칼럼 수, 1

  X_test = df_scaled_test.values
  X_test_t = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

  y_train = df_all[df_all['stn']==i]['uv'].values
  y_test = df[df['stn']==i]['uv'].values

  # LSTM 아키텍처 (architecture)
  model = Sequential()
  model.add(LSTM(units = 50, input_shape=(len(scale_cols), 1),activation='relu')) 
  model.add(Dense(1)) # 마지막 output 값이 하나

  # 컴파일링
  model.compile(optimizer='adam', loss='mse')

  # training data 세트에 피팅하기
  early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)
  model.fit(X_train_t, y_train, epochs = 10, batch_size = 120, verbose = 1, callbacks=[early_stop])

  #X_test_t를 LSTM 모델에 넣어서 예측하기
  y_pred = model.predict(X_test_t)

  #uv 값이 0보다 작은 건 0으로 바꾸기
  for i in range(len(y_pred)):
    if y_pred[i][0]<0:
      y_pred[i][0] = 0
  
  # 예측 배열에 넣어주기
  prediction.append(y_pred)

  # 모델 성능 평가하기
  predict_value = pd.DataFrame(y_pred)
  real_value = pd.DataFrame(y_test)
  result = pd.concat([real_value, predict_value],axis = 1)
  result.columns = ['real','predict']
  count = 0
  for i in range(len(result)):
    if abs(result['real'][i] - result['predict'][i]) <= DIFF:
      count+=1
  TP.append(round(count/len(result),3))



Epoch 1/10
831/831 [==============================] - 11s 11ms/step - loss: 2.1364
Epoch 2/10
831/831 [==============================] - 9s 11ms/step - loss: 0.5032
Epoch 3/10
831/831 [==============================] - 9s 11ms/step - loss: 0.4051
Epoch 4/10
831/831 [==============================] - 9s 11ms/step - loss: 0.3801
Epoch 5/10
831/831 [==============================] - 10s 12ms/step - loss: 0.3697
Epoch 6/10
831/831 [==============================] - 9s 11ms/step - loss: 0.3615
Epoch 7/10
831/831 [==============================] - 9s 11ms/step - loss: 0.3531
Epoch 8/10
831/831 [==============================] - 9s 11ms/step - loss: 0.3487
Epoch 9/10
831/831 [==============================] - 10s 12ms/step - loss: 0.3491
Epoch 9: early stopping
Epoch 1/10
831/831 [==============================] - 10s 11ms/step - loss: 1.6342
Epoch 2/10
831/831 [==============================] - 9s 11ms/step - loss: 0.4786
Epoch 3/10
831/831 [==============================] - 8s 10ms/step - l

In [6]:
for i in range(len(TP)) :
  print('stn 값 ',stn_index[i],': ',TP[i])

stn 값  13 :  0.552
stn 값  105 :  0.534
stn 값  108 :  0.546
stn 값  112 :  0.529
stn 값  115 :  0.527
stn 값  131 :  0.505
stn 값  132 :  0.505
